In [2]:
%%sh
## Dependencies
pip install pydot
pip install pydotplus
pip install tqdm

  Created wheel for pydotplus: filename=pydotplus-2.0.2-cp36-none-any.whl size=23674 sha256=fa9d5e1fb028e2eec150abc4ea57d84a21a7975853ffd4fa3ed39f6516f58c4d
  Stored in directory: /home/ec2-user/.cache/pip/wheels/35/7b/ab/66fb7b2ac1f6df87475b09dc48e707b6e0de80a6d8444e3628
Successfully built pydotplus
  Using cached https://files.pythonhosted.org/packages/4a/1c/6359be64e8301b84160f6f6f7936bbfaaa5e9a4eab6cbc681db07600b949/tqdm-4.45.0-py2.py3-none-any.whl


You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
%load_ext autoreload
%autoreload 2

# helper python file
import ml_pipeline as pipeline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Using TensorFlow backend.


In [1]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Input, LSTM, Bidirectional, Conv1D
from keras.layers import Dropout, Embedding
from keras.preprocessing import text, sequence
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D
from keras.models import Model

Using TensorFlow backend.


In [64]:
(train_df,test_df)=pipeline.read_input_data()
(seq_test,seq_train,word_index)=pipeline.tokenize_data(test_df,train_df)

  2%|▏         | 3705/153164 [00:00<00:04, 37045.80it/s]

num train:  159571
num test:  153164
pre-processing train data...


100%|██████████| 159571/159571 [00:04<00:00, 34828.63it/s]


tokenizing input data...
dictionary size:  282101


In [7]:
EMBEDDING_FILE = 'Glove-Twitter'
embedding_matrix=pipeline.prepare_embeddings(EMBEDDING_FILE,word_index)

1193514it [07:59, 2488.03it/s]


preparing embedding matrix...
number of null word embeddings: 2310


In [16]:
max_features=30000
max_seq_len=150

In [28]:
#import the required libraries
import keras
from keras.layers import Embedding
from keras.layers import Dense, Flatten, LSTM
from keras.layers import Input, GlobalMaxPool1D, Dropout, SpatialDropout1D, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers import Activation
from keras.layers import Bidirectional
from keras.layers import BatchNormalization
from keras.models import Model, Sequential
from keras import optimizers
from keras.layers import concatenate

import os
import pandas as pd

In [20]:
def init_BiLSTM_Conv_Model(embedding_matrix):
    # Build Model
    inp = Input(shape=(max_seq_len,))

    x = Embedding(max_features, embedding_matrix.shape[1], weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(0.35)(x)

    x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.15, recurrent_dropout=0.15))(x)
    x = Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)

    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool])

    out = Dense(6, activation='sigmoid')(x)

    model = Model(inp, out)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

In [65]:
model=init_BiLSTM_Conv_Model(embedding_matrix)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 150, 200)     6000000     input_10[0][0]                   
__________________________________________________________________________________________________
spatial_dropout1d_8 (SpatialDro (None, 150, 200)     0           embedding_10[0][0]               
__________________________________________________________________________________________________
bidirectional_8 (Bidirectional) (None, 150, 256)     336896      spatial_dropout1d_8[0][0]        
__________________________________________________________________________________________________
conv1d_7 (

In [66]:
label_names = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

y_train = train_df[label_names].values
history = model.fit(seq_train, y_train, epochs = 5, batch_size = 128, validation_split = 0.2)

Train on 127656 samples, validate on 31915 samples
Epoch 1/5
127656/127656 [==============================] - 541s 4ms/step - loss: 0.0627 - acc: 0.9782 - val_loss: 0.0468 - val_acc: 0.9826
Epoch 2/5
127656/127656 [==============================] - 536s 4ms/step - loss: 0.0482 - acc: 0.9818 - val_loss: 0.0451 - val_acc: 0.9826
Epoch 3/5
127656/127656 [==============================] - 539s 4ms/step - loss: 0.0452 - acc: 0.9827 - val_loss: 0.0439 - val_acc: 0.9830
Epoch 4/5
127656/127656 [==============================] - 539s 4ms/step - loss: 0.0434 - acc: 0.9833 - val_loss: 0.0424 - val_acc: 0.9837
Epoch 5/5
127656/127656 [==============================] - 539s 4ms/step - loss: 0.0415 - acc: 0.9839 - val_loss: 0.0415 - val_acc: 0.9838


In [67]:
# serialize model to JSON
model_json = model.to_json()
with open("Bi-LSTM-Conv-Glove-Preprocessed-model-twitter.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("Bi-LSTM-Conv-Glove-Preprocessed-model-twitter.h5")
print("Saved model to disk")

Saved model to disk


In [68]:
# Get predictions
predictions = model.predict(seq_test, batch_size=128, verbose=1)

submission = pd.read_csv('./data/sample_submission.csv')
submission[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']] = predictions
submission.to_csv('submission-Bi-LSTM-Conv-Glove-Preprocessed-model-twitter.csv', index=False)

153164/153164 [==============================] - 209s 1ms/step


## Pseudo-Labeling

1. Merge the predictions on 50% of the test data with the training data.
2. Retrain the model on this new dataset.

In [69]:
#Normalize the submissions
submission_norm = submission.copy()
submission_norm['toxic'] = submission['toxic'].apply(lambda x: 0 if x < 0.5 else 1)
submission_norm['severe_toxic'] = submission['severe_toxic'].apply(lambda x: 0 if x < 0.5 else 1)
submission_norm['obscene'] = submission['obscene'].apply(lambda x: 0 if x < 0.5 else 1)
submission_norm['threat'] = submission['threat'].apply(lambda x: 0 if x < 0.5 else 1)
submission_norm['insult'] = submission['insult'].apply(lambda x: 0 if x < 0.5 else 1)
submission_norm['identity_hate'] = submission['identity_hate'].apply(lambda x: 0 if x < 0.5 else 1)
submission_norm.head(5)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,1,0,1,0,1,0
1,0000247867823ef7,0,0,0,0,0,0
2,00013b17ad220c46,0,0,0,0,0,0
3,00017563c3f7919a,0,0,0,0,0,0
4,00017695ad8997eb,0,0,0,0,0,0


In [70]:
#Combine the predictons with the test data to create Pseudo Labeled Data
test_x=pd.concat([test_df['comment_text'], submission_norm], axis=1, sort=False)
test_x.head(5)

,comment_text,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,yo bitch ja rule is more succesful then you'll...,00001cee341fdb12,1,0,1,0,1,0
1,from rfc the title is fine as it is imo,0000247867823ef7,0,0,0,0,0,0
2,sources zawe ashton on lapland,00013b17ad220c46,0,0,0,0,0,0
3,if you have a look back at the source the info...,00017563c3f7919a,0,0,0,0,0,0
4,i don't anonymously edit articles at all,00017695ad8997eb,0,0,0,0,0,0


In [71]:
#take random 50% sample of the DataFrame with replacement
test_x=test_x.sample(frac=0.5, replace=True, random_state=1)

In [72]:
#merge with training dataset
combined=pd.concat([test_x,train_df])
print (combined.count())
combined.head(5)

comment_text     236153
id               236153
identity_hate    236153
insult           236153
obscene          236153
severe_toxic     236153
threat           236153
toxic            236153
dtype: int64


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


,comment_text,id,identity_hate,insult,obscene,severe_toxic,threat,toxic
128037,hey don't worry about it there's all manner of...,d5f8dbc8d5a2138f,0,0,0,0,0,0
5192,stylistics thanks for your email i'm glad you ...,08bdc11d90732729,0,0,0,0,0,0
50057,sources for stuff opposed to ayers i found an ...,531f1a43c0816123,0,0,0,0,0,0
109259,there is a probably the most complete to date ...,b64616aa8465aa99,0,0,0,0,0,0
73349,unsourced statement controls no territory in t...,7a3f64a479224faf,0,0,0,0,0,0


In [73]:
combined['comment_text']=combined.comment_text.astype(str)
combined['comment_text'].fillna(' ')
combined_train_y = combined[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values
combined_train_x = combined['comment_text'].str.lower()

In [74]:
(seq_train,seq_test,word_index)=pipeline.tokenize_data(combined,test_df)

  2%|▏         | 3558/236153 [00:00<00:06, 35550.28it/s]

pre-processing train data...


100%|██████████| 153164/153164 [00:03<00:00, 38430.59it/s]


tokenizing input data...
dictionary size:  282101


In [75]:
model=init_BiLSTM_Conv_Model(embedding_matrix)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, 150, 200)     6000000     input_11[0][0]                   
__________________________________________________________________________________________________
spatial_dropout1d_9 (SpatialDro (None, 150, 200)     0           embedding_11[0][0]               
__________________________________________________________________________________________________
bidirectional_9 (Bidirectional) (None, 150, 256)     336896      spatial_dropout1d_9[0][0]        
__________________________________________________________________________________________________
conv1d_8 (

In [ ]:
# Train the model
batch_size = 32
epochs = 5

model.fit(seq_train, combined_train_y, batch_size=batch_size, epochs=epochs, verbose=1,validation_split = 0.2)

Train on 188922 samples, validate on 47231 samples
Epoch 1/5
188922/188922 [==============================] - 3344s 18ms/step - loss: 0.0830 - acc: 0.9725 - val_loss: 0.0552 - val_acc: 0.9818
Epoch 2/5
188922/188922 [==============================] - 3336s 18ms/step - loss: 0.0579 - acc: 0.9798 - val_loss: 0.0526 - val_acc: 0.9822
Epoch 3/5
188922/188922 [==============================] - 3337s 18ms/step - loss: 0.0530 - acc: 0.9813 - val_loss: 0.0529 - val_acc: 0.9824
Epoch 4/5
188922/188922 [==============================] - 3331s 18ms/step - loss: 0.0502 - acc: 0.9823 - val_loss: 0.0524 - val_acc: 0.9823
Epoch 5/5
188896/188922 [============================>.] - ETA: 0s - loss: 0.0483 - acc: 0.9830

In [ ]:
# Save final model
# serialize model to JSON
model_json = model.to_json()
with open("Bi-LSTM-Conv-Glove-Preprocessed-model-twitter-pseudolabel.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("Bi-LSTM-Conv-Glove-Preprocessed-model-twitter-pseudolabel.h5")
print("Saved model to disk")

In [ ]:
# Generate final predictions
predictions = model.predict(seq_test, batch_size=batch_size, verbose=1)

submission = pd.read_csv('./data/sample_submission.csv')
submission[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']] = predictions
submission.to_csv('submission-Bi-LSTM-Conv-Glove-Twitter-Plabel.csv', index=False)